In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Input, Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPool2D
from matplotlib import pyplot as plt

In [ ]:
!curl -sO https://raw.githubusercontent.com/mbrukman/reimplementing-ml-papers/main/alexnet/local_response_normalization.py

from local_response_normalization import LocalResponseNormalization

In [ ]:
# Load the CIFAR-10 dataset.
(x_train_raw, y_train_raw), (x_test_raw, y_test_raw) = keras.datasets.cifar10.load_data();

170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
num_classes = 10

# Scale train and test inputs by converting them from range of [0, 255] to [0.0, 1.0]
x_train = x_train_raw.astype('float32') / 255.0
x_test = x_test_raw.astype('float32') / 255.0

# Convert the output to categorical one-hot encoding to match the output of our
# network.
y_train = keras.utils.to_categorical(y_train_raw, num_classes)
y_test = keras.utils.to_categorical(y_test_raw, num_classes)

In [ ]:
# Define the model architecture, this comes from the the Google Code project
# mentioned in the paper — https://code.google.com/p/cuda-convnet/ —
# specifically, the file `examples-layers/layers-conv-local-11pct.cfg`.
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu', name='Conv1'),
    MaxPool2D(pool_size=3, strides=2, padding='valid', name='MaxPool1'),
    LocalResponseNormalization(name='LRN'),
    Conv2D(filters=64, kernel_size=(5, 5), padding='same', activation='relu', name='Conv2'),
    LocalResponseNormalization(name='LRN2'),
    MaxPool2D(pool_size=3, strides=2, padding='valid', name='MaxPool2'),
    Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', name='Local3'),
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', name='Local4'),
    Flatten(),
    Dense(10, activation='softmax', name="FC10"),
], name="CIFAR-10-TF-NN-LRN")

model.summary()

Model: "CIFAR-10-TF-NN-LRN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 32, 32, 64)        4864      
                                                                 
 MaxPool1 (MaxPooling2D)     (None, 15, 15, 64)        0         
                                                                 
 LRN (LocalResponseNormaliza  (None, 15, 15, 64)       0         
 tion)                                                           
                                                                 
 Conv2 (Conv2D)              (None, 15, 15, 64)        102464    
                                                                 
 LRN2 (LocalResponseNormaliz  (None, 15, 15, 64)       0         
 ation)                                                          
                                                                 
 MaxPool2 (MaxPooling2D)     (None, 7, 7, 64)   

In [ ]:
# Compile the model with optimizer and loss function.
opt = keras.optimizers.Adam(learning_rate=0.001)
loss_fn = keras.losses.CategoricalCrossentropy()
model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=20)

Epoch 1/20
1563/1563 [==============================] - 349s 223ms/step - loss: 1.5501 - accuracy: 0.4335
Epoch 2/20
1563/1563 [==============================] - 346s 222ms/step - loss: 1.1709 - accuracy: 0.5857
Epoch 3/20
1563/1563 [==============================] - 346s 221ms/step - loss: 1.0080 - accuracy: 0.6451
Epoch 4/20
1563/1563 [==============================] - 354s 226ms/step - loss: 0.9025 - accuracy: 0.6832
Epoch 5/20
1563/1563 [==============================] - 347s 222ms/step - loss: 0.8180 - accuracy: 0.7159
Epoch 6/20
1563/1563 [==============================] - 342s 219ms/step - loss: 0.7575 - accuracy: 0.7365
Epoch 7/20
1563/1563 [==============================] - 342s 219ms/step - loss: 0.7000 - accuracy: 0.7549
Epoch 8/20
1563/1563 [==============================] - 344s 220ms/step - loss: 0.6443 - accuracy: 0.7763
Epoch 9/20
1563/1563 [==============================] - 345s 221ms/step - loss: 0.6026 - accuracy: 0.7881
Epoch 10/20
1563/1563 [=======================

In [ ]:
# Evaluate the model
model.evaluate(x_test, y_test)

313/313 [==============================] - 19s 60ms/step - loss: 1.2247 - accuracy: 0.7003


[1.2246522903442383, 0.7002999782562256]